MTG Judge RAG
---------------------------------------------------
This is a simple Python script for building an AI-powered MTG rules assistant
using Retrieval-Augmented Generation (RAG) with OpenAI + FAISS.

- Loads the Comprehensive Rules from a text file.
- Splits rules into chunks.
- Creates embeddings with OpenAI.
- Stores them in ChromaDB for fast search (not using FAISS due to py versioning)
- Lets you ask questions, retrieves relevant rules, and asks the LLM to answer.

In [94]:
# -------- IMPORTS --------
import os
import re
import json
import chromadb

from openai import OpenAI
# client = OpenAI(api_key=OPENAI_API_KEY)   # don’t pass api_key explicitly

from dotenv import load_dotenv


In [95]:
# -------- CONFIG --------
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
EMBED_MODEL = "text-embedding-3-large"
CHAT_MODEL = "gpt-4o-mini"
CHROMA_DB_DIR = "./chroma_db"
os.makedirs(CHROMA_DB_DIR, exist_ok=True) # to create folder if it doesn't exist
RULES_FILE = "./comprehensive-rules.txt"
CARDS_FILE = "./clean-standard-cards.json"
CHUNK_SIZE = 700 # words approximation
TOP_K = 6

In [96]:
# -------- INITIALIZATION --------
client = OpenAI(api_key=OPENAI_API_KEY)
load_dotenv()

True

In [97]:
# -------- HELPER LOAD RULES --------
def load_rules(path):
    """Load the MTG comprehensive rules from a text file."""
    if not os.path.exists(path):
        print(f"Rules file not found at {path}")
        return []

    docs = []
    with open(path, "r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            # Rules usually like: 603.1. Some text
            match = re.match(r"^(\d{1,3}(?:\.\d+)+)\s+(.*)$", line)
            if match:
                rule_id, body = match.groups()
                docs.append({
                    "id": f"CR:{rule_id}",
                    "text": f"{rule_id} {body}",
                    "rule_id": rule_id,
                    "source": "Comprehensive Rules"
                })
    return docs

In [99]:
# -------- HELPER LOAD CARDS --------
def load_cards(path):
    """Load MTG card data from your JSON export."""
    if not os.path.exists(path):
        print(f"Card file not found at {path}")
        return []

    with open(path, "r", encoding="utf-8") as f:
        cards = json.load(f)

    docs = []
    for c in cards:
        # Skip cards without names or text
        if "name" not in c or not c.get("originalText"):
            continue

        # Construct a searchable text block for embedding
        text_parts = [
            f"Name: {c['name']}",
            f"Mana Cost: {c.get('manaCost', '')}",
            f"Types: {' '.join(c.get('types', []))}",
            f"Subtypes: {' '.join(c.get('subtypes', []))}",
            f"Abilities/Keywords: {', '.join(c.get('keywords', []))}",
            f"Text: {c['originalText']}"
        ]

        # Add rulings (big chunk but useful)
        rulings = c.get("rulings", [])
        if rulings:
            rulings_text = " | ".join(r["text"] for r in rulings if "text" in r)
            text_parts.append(f"Rulings: {rulings_text}")

        full_text = "\n".join(text_parts)

        docs.append({
            "id": f"CARD:{c['uuid']}",   # use UUID for uniqueness
            "text": full_text,
            "source": "Card Database",
            "card_name": c["name"],
            "manaCost": c.get("manaCost", ""),
            "types": ", ".join(c.get("types", [])),       # FIXED: stringify list
            "subtypes": ", ".join(c.get("subtypes", [])), # FIXED: stringify list
            "keywords": ", ".join(c.get("keywords", [])), # FIXED: stringify list
            "rarity": c.get("rarity", "")
        })

    print(f"Loaded {len(docs)} cards from {path}")
    return docs


In [100]:
# -------- HELPER CHUNK TEXT --------
def chunk_text(text, chunk_size=CHUNK_SIZE):
    """Split text into smaller chunks so embeddings don't get too big."""
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks = []
    current = []
    length = 0

    for s in sentences:
        tokens = len(s.split())
        if length + tokens > chunk_size:
            chunks.append(" ".join(current))
            current = [s]
            length = tokens
        else:
            current.append(s)
            length += tokens
    if current:
        chunks.append(" ".join(current))

    return chunks

In [101]:
# -------- HELPER BUILD INDEX --------
def build_index():
    """Create ChromaDB collection from rules + card data."""
    client = OpenAI(api_key=OPENAI_API_KEY)

    print("Loading rules...")
    rules = load_rules(RULES_FILE)

    print("Loading cards...")
    cards = load_cards(CARDS_FILE)  # add this

    all_docs = rules + cards  # merge datasets

    texts, metas, ids = [], [], []

    for d in all_docs:
        chunks = chunk_text(d["text"])
        for i, ch in enumerate(chunks):
            texts.append(ch)
            metas.append(d)
            ids.append(f"{d['id']}_{i}")

    if not texts:
        raise ValueError("No valid chunks found to embed.")

    print(f"Total chunks: {len(texts)}")

    # Create embeddings
    embeddings = client.embeddings.create(model=EMBED_MODEL, input=texts)
    vecs = [d.embedding for d in embeddings.data]

    # Initialize Chroma client
    chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)

    # Drop old collection (clean rebuild)
    try:
        chroma_client.delete_collection("mtg_data")
    except:
        pass

    collection = chroma_client.get_or_create_collection(name="mtg_data")

    # Add to Chroma
    collection.add(
        ids=ids,
        embeddings=vecs,
        documents=texts,
        metadatas=metas
    )

    print("Index built and saved with ChromaDB!")




In [102]:
# -------- HELPER SEARCH INDEX --------
def search_index(query, top_k=TOP_K):
    """Search ChromaDB for relevant rule chunks."""
    query = query.strip()
    if not query:
        raise ValueError("Empty query provided.")

    # client = OpenAI()
    emb = client.embeddings.create(model=EMBED_MODEL, input=[query])
    vec = emb.data[0].embedding

    chroma_client = chromadb.PersistentClient(path=CHROMA_DB_DIR)
    collection = chroma_client.get_or_create_collection(name="mtg_rules")

    results = collection.query(query_embeddings=[vec], n_results=top_k)

    docs = []
    for i, doc in enumerate(results["documents"][0]):
        docs.append({
            "text": doc,
            "meta": results["metadatas"][0][i]
        })
    return docs

In [103]:
# -------- HELPER GENERATE SUBQUERIES --------
def generate_subqueries(query, n=10):
    """Chain of Thought decomposition function. Use the LLM to break a user query into smaller sub-questions."""
    #client = OpenAI(api_key=OPENAI_API_KEY)
    prompt = f"""
    Break down the following Magic: The Gathering rules question into {n} smaller, 
    more specific sub-questions that cover timing, abilities, rules interactions, 
    and possible edge cases. Return them as a numbered list.

    Original Question: {query}
    """
    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0.2,
        messages=[
            {"role": "system", "content": "You are an expert MTG judge assistant."},
            {"role": "user", "content": prompt}
        ]
    )
    text = resp.choices[0].message.content
    subqueries = [line.strip("0123456789. ") for line in text.splitlines() if line.strip()]
    return subqueries

In [ ]:
# -------- HELPER ANSWER WITH SUBQUERIES --------
def answer_with_subqueries(query):
    """Break question into subqueries, search index for each, and generate final answer."""
    # Step 1: Get subqueries
    subqueries = generate_subqueries(query, n=10)
    # print("Subqueries:", subqueries)

    # Step 2: Collect context from all subqueries
    all_context = []
    for sq in subqueries:
        results = search_index(sq, top_k=5)  # use your existing search_index
        for i, r in enumerate(results, 1):
            all_context.append(f"Subquery: {sq}\n- Source: {r['meta'].get('source', '')}\n- Text: {r['text']}")

    context = "\n\n".join(all_context)

    response_format = """
    Please provide a structured answer with the following format:
    - Question: [a rephrased version of the user question in the most clear form]
    - Short Answer: [short summary answer, this sentence should always start with a Yes or No if possible]
    - Full Explanation: [detailed reasoning with rules and card interactions]
    - Sources: [cite the rules used for the response and also add the full text of the cards involved or that were most relevant]
    """

    #! temporal reply format for benchmarks
    # response_format = "Respond only with a Yes or No answer. Nothing more should be added"

    # Step 3: Ask LLM for final structured response
    system_prompt = f"""
    You are an expert Magic: The Gathering judge assistant.
    A user has a question about card interactions or rules.

    Use these sources (rules + card texts):
    {context}

    Answer format:
    {response_format}
    """

    user_prompt = f"""
    The question from the user is:
    {query}
    """
    
    client = OpenAI(api_key=OPENAI_API_KEY)
    resp = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0, # lowering temperature for more accurate and consistant response according to rules
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )

    #* SECONDARY JUDGE VERIFICATION
    verification_system_prompt = f"""
    You are an expert Magic: The Gathering high judge.
    You will be given:

    - A user’s question
    - A judge’s ruling on that question
    - All the context provided by the judge

    Your task is to carefully review and analyze the information. Then, determine whether the context adequately supports the judge’s ruling. Based on your analysis, you may either accept or deny the ruling.

    Your response should follow this format:

    - If you agree with the ruling, respond: Accepted
    - If you disagree with the ruling, respond: Denied, [include context explaining the reason for denial]

    The original user question is:
    {query}
    """

    judge_prompt = f"""
    The context brought by the initial ruling judge was:
    {context}

    The final response from the initial judge was:
    {resp.choices[0].message.content}
    """

    client = OpenAI(api_key=OPENAI_API_KEY) # new model initialization for new judge
    resp2 = client.chat.completions.create(
        model=CHAT_MODEL,
        temperature=0, # lowering temperature for more accurate and consistant response according to rules
        messages=[
            {"role": "system", "content": verification_system_prompt},
            {"role": "user", "content": judge_prompt}
        ]
    )

    judge2_response = resp2.choices[0].message.content

    if judge2_response == "Accepted":
        return resp.choices[0].message.content
    else:
        return f"""
        We’re sorry, our virtual judges were unable to reach an agreement on a final response to your question.

        Reason: The original judge’s ruling was: {judge2_response}

        Please try asking your question again, this time with clearer wording to help us provide a more definitive answer.
        """


In [105]:
# -------- BUILDING INDEX --------
build_index()  # only first time

Loading rules...
Loading cards...
Loaded 90 cards from ./clean-standard-cards.json
Total chunks: 91
Index built and saved with ChromaDB!


# Single testing

In [108]:
single_question = "Someone is playing Flashfreeze on one of my spells, Can I play Aven Interrupter on top of Flashfreeze so my initial spell can be resolved?. Right answer: Yes. Response: No"
response = answer_with_subqueries(single_question)

In [110]:
print(response)

- Question: Can I play Aven Interrupter on top of Flashfreeze to allow my original spell to resolve?
- Short Answer: Yes.
- Full Explanation: In this scenario, you can respond to Flashfreeze by casting Aven Interrupter. Flashfreeze is an instant that counters a target spell that is blue, and it goes on the stack above the original spell. When you cast Aven Interrupter, it will also go on the stack above Flashfreeze. However, Aven Interrupter has the ability to counter a spell, but it can only do so if it resolves before Flashfreeze. If Aven Interrupter resolves successfully, it can counter Flashfreeze, allowing your original spell to resolve. If Flashfreeze resolves first, it will counter your original spell. Therefore, the timing of casting Aven Interrupter is crucial, and you must ensure it resolves before Flashfreeze does.
- Sources: Comprehensive Rules, specifically regarding the stack and spell resolution.


# Multiple testing

In [82]:
with open("easy-questions.json", "r", encoding="utf-8") as f:
    easy_questions = json.load(f)
with open("hard-questions.json", "r", encoding="utf-8") as f:
    hard_questions = json.load(f)
with open("own-questions.json", "r", encoding="utf-8") as f:
    own_questions = json.load(f)

all_questions = easy_questions + hard_questions + own_questions

correct_answers = 0
wrong_answered_questions = []

for question in all_questions:
    # response = answer_question(question["text"])
    response = answer_with_subqueries(question["text"])
    # response = answer_with_subqueries(question.text)
    if response == question["answer"]:
        correct_answers += 1
    else:
        wrong_answered_questions.append(f"{question["text"]}. Right answer: {question["answer"]}. Response: {response}")

print(f"correct answers: {correct_answers}/{len(all_questions)}")
for each_wrong_question in wrong_answered_questions:
    print(each_wrong_question)


correct answers: 39/45
Does scry let you look at cards and choose to put some on top or bottom of your library?. Right answer: Yes. Response: No
Can there be infinite or multiple cleanup steps triggered by effects like Kozilek plus discard effects?. Right answer: Yes. Response: No
Are continuous effects applied in a specific layered system, such as type-changing, ability additions, P/T changes, in numbered order?. Right answer: Yes. Response: No
Are special actions like playing a land or turning a face-down creature face-up things opponents cannot respond to?. Right answer: Yes. Response: No
If I imprint Time Walk on Panoptic Mirror, do I get infinite turns?. Right answer: Yes. Response: No
Someone is playing Flashfreeze on one of my spells, Can I play Aven Interrupter on top of Flashfreeze so my initial spell can be resolved?. Right answer: Yes. Response: No


# Results



## Test: No temperature set and no subqueries.

### Correct answers: 30/45

## Test: Temperature set to 0.1 and no subqueries.

### Correct answers: 35/45

## Test: Temperature set to 0.1 and 5 subqueries.

### Correct answers: 37/45

#### Incorrect answer to:
- Does scry let you look at cards and choose to put some on top or bottom of your library?
- If you draw more than seven cards, can you keep them all if no other effect limits your hand size?
- Can there be infinite or multiple cleanup steps triggered by effects like Kozilek plus discard effects?
- If a creature phases out and back in, does it lose summoning sickness if it had it before?
- Are special actions like playing a land or turning a face-down creature face-up things opponents cannot respond to?
- If I imprint Time Walk on Panoptic Mirror, do I get infinite turns?
- I am being attacked by Axebane Ferox and I declare Aegis Turtle as a blocker. But before assigning damage, I play Bounce Off on Aegis Turtle. Do I still receive 4 damage from Agonasaur Rex?
- Someone is playing Flashfreeze on one of my spells, Can I play Aven Interrupter on top of Flashfreeze so my initial spell can be resolved?

## Test: Temperature set to 0 and 10 subqueries.

- 8 minutes for 45 queries (10 seconds per query)

### Correct answers: 39/45

#### Incorrect answer to:
- Does scry let you look at cards and choose to put some on top or bottom of your library?. Right answer: Yes. Response: No
- Can there be infinite or multiple cleanup steps triggered by effects like Kozilek plus discard effects?. Right answer: Yes. Response: No
- Are continuous effects applied in a specific layered system, such as type-changing, ability additions, P/T changes, in numbered order?. Right answer: Yes. Response: No
- Are special actions like playing a land or turning a face-down creature face-up things opponents cannot respond to?. Right answer: Yes. Response: No
- If I imprint Time Walk on Panoptic Mirror, do I get infinite turns?. Right answer: Yes. Response: No
- Someone is playing Flashfreeze on one of my spells, Can I play Aven Interrupter on top of Flashfreeze so my initial spell can be resolved?. Right answer: Yes. Response: No